# Explicit Features

In [1]:
import json

from warofwords import TrainedWarOfWordsLatent

LEG = 8


def sort_params(group, parameters, features, reverse=True, n=None):
    params = [
        (feat, parameters[feat])
        for feat in features.get_group(group, by_feature_name=True)
    ]
    if n is None:
        n = len(params)
    return sorted(params, key=lambda p: p[1], reverse=reverse)[:n]


def print_params(group, parameters, features, n=None):
    print('###', group.upper(), '\n')
    sortedparams = sort_params(group, parameters, features, n=n)
    for name, p in sortedparams:
        print(f'{p:+.2f} {name}')
    print()


def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

## Load data, parameters, and features

In [2]:
# Load mapping from dossier to title.
titles = load_json('../data/helpers/dossier-titles.json')
# Load mapping from dossier to title.
meps = load_json('../data/helpers/meps.json')

# Load trained model to get parameters and features.
models = '../2-training/trained-models'
model = f'{models}/ep{LEG}-all_features-latent-text.fit'
model = TrainedWarOfWordsLatent.load(model)
parameters = model.parameters
features = model.features

## Analyze MEP, edit, and dossier features

In [3]:
# Define groups of features to analyze.
groups = [
    # MEP features.
    'political-group',
    'nationality',
    'gender',
    'rapporteur',
    # Edit features.
    'edit-length',
    'justification',
    'outsider',
    'article-type',
    'edit-type',
    # Dossier features.
    'dossier-type',
    'legal-act',
    'committee',
]
for group in groups:
    print_params(group, parameters, features)

### POLITICAL-GROUP 

+0.33 Group of the European People's Party
+0.30 Group of the Progressive Alliance of Socialists and Democrats in the European Parliament
+0.29 Group of the Alliance of Liberals and Democrats for Europe
+0.25 Europe of Freedom and Direct Democracy Group
+0.16 Group of the Greens/European Free Alliance
-0.02 European Conservatives and Reformists Group
-0.05 Confederal Group of the European United Left
-0.33 Non-attached Members
-1.02 Europe of Nations and Freedom Group

### NATIONALITY 

+0.75 Latvia
+0.50 Finland
+0.44 Slovakia
+0.39 Slovenia
+0.35 Malta
+0.31 Hungary
+0.30 Luxembourg
+0.30 Romania
+0.11 Netherlands
+0.08 Denmark
+0.06 Germany
+0.05 Croatia
+0.04 Czech Republic
+0.03 Spain
+0.02 Austria
+0.02 Poland
-0.00 Ireland
-0.01 Portugal
-0.03 Estonia
-0.05 Cyprus
-0.05 Lithuania
-0.07 Greece
-0.07 Bulgaria
-0.08 Italy
-0.10 United Kingdom
-0.12 France
-0.13 Belgium
-0.21 Sweden

### GENDER 

-0.02 F
-0.04 M

### RAPPORTEUR 

+1.19 rapporteur

### EDIT-LENG

## Analyze dossier controversy

In [4]:
# Print top-10 and bottom-10 dossiers.
print('### TOP-10 DOSSIERS\n')
header = f'PARAM | {"DOSSIER":<19} | TITLE'
print(header)
print('-' * len(header))
dossiers = sort_params('dossier', parameters, features, reverse=True, n=10)
for doss, p in dossiers:
    title = titles[doss]
    print(f'{p:+.2f} | {doss:<19} | {title}')
print('\n### BOTTOM-10 DOSSIERS\n')
dossiers = sort_params('dossier', parameters, features, reverse=False, n=10)
for doss, p in dossiers:
    title = titles[doss]
    print(f'{p:+.2f} | {doss:<19} | {title}')

### TOP-10 DOSSIERS

PARAM | DOSSIER             | TITLE
-----------------------------------
+2.02 | A8-0198-2018        | Establishing a framework for screening of foreign direct investments into the European Union
+1.96 | ITRE-AD(2016)582103 | Cost-effective emission reductions and low-carbon investments
+1.88 | PETI-AD(2019)629635 | Discontinuing seasonal changes of time
+1.62 | A8-0289-2018        | Proposal for a regulation of the European Parliament and of the Council on health technology assessment and amending Directive 2011/24/EU
+1.51 | A8-0013-2019        | Regulation amending Regulations establishing European Supervisory Authorities and Regulations on financial markets: (EU) No 1093/2010, (EU) No 1094/2010, (EU) No 1095/2010, (EU) No 345/2013, (EU) No 346/2013, (EU) No 600/2014, (EU) 2015/760, (EU) 2016/1011 and (EU) 2017/1129
+1.45 | A8-0482-2018        | Establishing the InvestEU Programme
+1.39 | A8-0401-2018        | Proposal for a regulation of the European Parliament 

## Analyze MEP influence

In [5]:
# Print top-10 MEPs.
print('\n### MEPS\n')
mep_ids = sort_params('mep', parameters, features, n=10)
header = f'PARAM | {"NAME":<20} | {"NATIONALITY":<15} | POLITICAL GROUP'
print(header)
print('-' * len(header))
for mep_id, p in mep_ids:
    mep_id = str(mep_id)
    name = meps[mep_id]['name']
    grp = meps[mep_id][f'group-ep{LEG}']
    nationality = meps[mep_id]['nationality']
    print(f'{p:+.2f} | {name:<20} | {nationality:<15} | {grp}')


### MEPS

PARAM | NAME                 | NATIONALITY     | POLITICAL GROUP
----------------------------------------------------------------
+1.65 | Steven WOOLFE        | United Kingdom  | Non-attached Members
+1.54 | Janusz KORWIN-MIKKE  | Poland          | Non-attached Members
+1.51 | Manuel dos SANTOS    | Portugal        | Group of the Progressive Alliance of Socialists and Democrats in the European Parliament
+1.39 | Pedro SILVA PEREIRA  | Portugal        | Group of the Progressive Alliance of Socialists and Democrats in the European Parliament
+1.27 | Laurenţiu REBEGA     | Romania         | Europe of Nations and Freedom Group
+1.25 | Knut FLECKENSTEIN    | Germany         | Group of the Progressive Alliance of Socialists and Democrats in the European Parliament
+1.25 | Arnaud DANJEAN       | France          | Group of the European People's Party
+1.20 | Ioan Mircea PAŞCU    | Romania         | Group of the Progressive Alliance of Socialists and Democrats in the European Parliam

## Print parameters of text embeddings

In [6]:
# Print text embeddings.
group = 'title-embedding'
print('\n###', group.upper(), '\n')
print(parameters.get_group(group))
group = 'edit-embedding'
print('\n###', group.upper(), '\n')
print(parameters.get_group(group))


### TITLE-EMBEDDING 

[-0.13636603951454163, 0.16064025461673737, 0.1437772959470749, -0.04790971800684929, -0.06381064653396606, -0.13413335382938385, 0.07477857172489166, 0.09877929836511612, 0.016619667410850525, 0.10573480278253555]

### EDIT-EMBEDDING 

[11.98774528503418, -13.799277305603027, -6.817662239074707, 5.438927173614502, 3.900665521621704, 8.195265769958496, -1.1281416416168213, -0.23749855160713196, -3.7958450317382812, -9.426227569580078]
